In [3]:
import pandas as pd
import geopandas
from shapely.geometry import Point
from shapely.ops import nearest_points
from math import *
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
fare_arrests = pd.read_csv('../data/Fare Evasion Arrests.csv')

In [5]:
stations = geopandas.read_file('../data/Subway Stations/geo_export_d1cef8bd-bb80-491b-983e-fc4b3e3aab4f.shp')

In [6]:
fare_arrests.columns

Index(['ARREST_KEY', 'ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD', 'OFNS_DESC',
       'LAW_CODE', 'LAW_CAT_CD', 'ARREST_BORO', 'ARREST_PRECINCT',
       'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'X_COORD_CD',
       'Y_COORD_CD', 'Latitude', 'Longitude'],
      dtype='object')

In [7]:
#original datasets have no nans
len(fare_arrests.dropna())- len(fare_arrests)

0

In [8]:
arrests_gdf = geopandas.GeoDataFrame(fare_arrests[['ARREST_KEY', 'ARREST_DATE', 'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE','Latitude', 'Longitude']], geometry= [Point(x, y) for x, y in zip(fare_arrests.Longitude, fare_arrests.Latitude)])

In [9]:
unary = stations.geometry.unary_union

In [10]:
def nearest_station(point, pts=unary):
     # find the nearest station and return it and the corresponding distance to that station
    nearest = stations.geometry == nearest_points(point, pts)[1]
    
    
    longs = stations[nearest].iloc[0].geometry.x
    lats = stations[nearest].iloc[0].geometry.y
    longp = point.x
    latp = point.y
    
    return [stations[nearest].iloc[0], haversine(longs, lats, longp, latp)]

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in km. Use 3956 for miles
    
    return c * r *1000 #conver to m

In [11]:
max_dist = 250

arrests_stations = pd.DataFrame()

for index, row in arrests_gdf.iterrows():
    [station, dist] = nearest_station(row.geometry)
    
    df = station[['line', 'name', 'notes', 'objectid']].copy()
    df['index'] = index
    
    if dist <= max_dist:
        arrests_stations = arrests_stations.append(df)

,line,name,notes,objectid
index,,,,
1.0,4-6-6 Express,77th St,"4 nights, 6-all times, 6 Express-weekdays AM s...",34.0
3.0,2-5,Prospect Ave,"2-all times, 5-all times exc nights, skips rus...",254.0
4.0,B-D,Kingsbridge Rd,"B-rush hours, D-all times",59.0
5.0,A-B-C-D,145th St,"A,D-all times, B-weekdays and evenings, C-all ...",295.0
6.0,A-C-E,W 4th St - Washington Sq (Upper),"A,E-all times, C-all times exc nights",206.0
...,...,...,...,...
292259.0,N-Q-R-W,49th St,"N-all times, Q-all times exc weekends, R-all t...",354.0
292260.0,L,8th Ave,L-all times,443.0
292261.0,B-D,Kingsbridge Rd,"B-rush hours, D-all times",59.0


In [12]:
print(len(arrests_stations))

268320


In [36]:
arrests_stations.index = arrests_stations.index.astype('int64')
print(arrests_stations.head())

                line                              name  \
index                                                    
1      4-6-6 Express                           77th St   
3                2-5                      Prospect Ave   
4                B-D                    Kingsbridge Rd   
5            A-B-C-D                          145th St   
6              A-C-E  W 4th St - Washington Sq (Upper)   

                                                   notes  objectid  
index                                                               
1      4 nights, 6-all times, 6 Express-weekdays AM s...      34.0  
3      2-all times, 5-all times exc nights, skips rus...     254.0  
4                              B-rush hours, D-all times      59.0  
5      A,D-all times, B-weekdays and evenings, C-all ...     295.0  
6                  A,E-all times, C-all times exc nights     206.0  


In [37]:
arrests_assigned = arrests_gdf.merge(arrests_stations, left_index=True, right_index = True)

In [38]:
print(arrests_assigned.head())

   ARREST_KEY ARREST_DATE ARREST_BORO  ARREST_PRECINCT  JURISDICTION_CODE  \
1   173113521  12/31/2017           M               19                1.0   
3   173114446  12/31/2017           B               40                1.0   
4   173093896  12/30/2017           B               52                1.0   
5   173102770  12/30/2017           M               30                1.0   
6   173107288  12/30/2017           M                6                1.0   

  AGE_GROUP PERP_SEX       PERP_RACE   Latitude  Longitude  \
1     25-44        F  WHITE HISPANIC  40.773650 -73.959857   
3     25-44        M  WHITE HISPANIC  40.819702 -73.901603   
4     25-44        M           BLACK  40.866076 -73.894376   
5     25-44        M           BLACK  40.824040 -73.944760   
6     45-64        M           BLACK  40.731715 -74.000958   

                                       geometry           line  \
1  POINT (-73.95985695199995 40.77365029300007)  4-6-6 Express   
3  POINT (-73.90160301699996 40.

In [39]:
print(len(arrests_assigned))

268320


In [41]:
#percent of arrests able to be attributible to stops w/ duplicates
100*len(arrests_assigned)/len(arrests_gdf)

91.8064646367876

In [43]:
#drop duplicates of arrests we couldn't uniquely attach to a station
arrests_assigned = arrests_assigned.drop_duplicates(subset='ARREST_KEY', keep = False)
print('Dropped: '+str(len(arrests_assigned) - len(arrests_stations))+ ' duplicates.')

Dropped: 0 duplicates.


In [44]:
#drop rows with any columns na (ie we could not assign to stations)
arrests_assigned = arrests_assigned.dropna()
print('Dropped: '+str(len(arrests_assigned) - len(arrests_stations))+ ' nans.')

Dropped: 0 nans.


In [45]:
arrests_assigned.to_csv('../data/Arrests_Assigned_250m.csv')

In [64]:
arrests_stations.groupby(['name', 'line']).count().reset_index().sort_values('objectid', ascending=False).drop('notes', axis=1).rename({'objectid': 'arrests'}, axis=1)

,name,line,arrests
96,42nd St - Port Authority Bus Term,A-C-E,12285
93,3rd Ave - 149th St,2-5,5955
17,125th St,4-5-6-6 Express,5176
11,116th St,4-6-6 Express,5101
36,161st St - Yankee Stadium,B-D,4979
...,...,...,...
302,Fulton St,J-Z,1
84,34th St - Hudson Yards,7-7 Express,1
386,New Utrecht Ave,N,1
106,50th St,D,1


In [65]:
arrests_assigned.head()

,ARREST_KEY,ARREST_DATE,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude,geometry,line,name,notes,objectid
1,173113521,12/31/2017,M,19,1.0,25-44,F,WHITE HISPANIC,40.773650,-73.959857,POINT (-73.95985695199995 40.77365029300007),4-6-6 Express,77th St,"4 nights, 6-all times, 6 Express-weekdays AM s...",34.0
3,173114446,12/31/2017,B,40,1.0,25-44,M,WHITE HISPANIC,40.819702,-73.901603,POINT (-73.90160301699996 40.81970204500004),2-5,Prospect Ave,"2-all times, 5-all times exc nights, skips rus...",254.0
4,173093896,12/30/2017,B,52,1.0,25-44,M,BLACK,40.866076,-73.894376,POINT (-73.89437578699994 40.86607576000005),B-D,Kingsbridge Rd,"B-rush hours, D-all times",59.0
5,173102770,12/30/2017,M,30,1.0,25-44,M,BLACK,40.824040,-73.944760,POINT (-73.94475961199998 40.82404000400004),A-B-C-D,145th St,"A,D-all times, B-weekdays and evenings, C-all ...",295.0
6,173107288,12/30/2017,M,6,1.0,45-64,M,BLACK,40.731715,-74.000958,POINT (-74.00095761299998 40.73171480100008),A-C-E,W 4th St - Washington Sq (Upper),"A,E-all times, C-all times exc nights",206.0
